# Multi i-TED characterization
## Fourth: Per pixel threshold

- In the [previous notebook](./PyROOT_resolution_Cs137v3.html) it was proposed to apply thresholds per pixel:
    - To get to a lower overall threshold that should equate to better resolution.
    - To decrease the size of files, which increases greatly for lower threshoilds due to noise.
    - To study if it impacts the considerable increase in resolution present in iTED-C-0 that was hypothesised being related to noise starting the integration window.
- The 888 configuration is compared to 887 and 88c which stands for 887 with some pixels changed manually.
- Studies of alpha activity and noise are added.

In [1]:
pkg_ver = lambda pkg: "{:<20}{:}".format(pkg.__name__,pkg.__version__)

# ROOT
import uproot
print(pkg_ver(uproot))
import ROOT

# Machine Learning
import sklearn
print(pkg_ver(sklearn))
import torch
print(pkg_ver(torch))

# Data science
import scipy
print(pkg_ver(scipy))
import numpy
print(pkg_ver(numpy))
import pandas
print(pkg_ver(pandas))

# Visualizations
import matplotlib
print(pkg_ver(matplotlib))
import matplotlib.pyplot as plt

import tqdm
print(pkg_ver(tqdm))

uproot              4.3.5
Welcome to JupyROOT 6.26/10
sklearn             1.2.2
torch               2.0.0rc5
scipy               1.10.1
numpy               1.23.5
pandas              1.5.3
matplotlib          3.6.3
tqdm                4.62.3


In [2]:
%jsroot

In [3]:
class spectrum:
    
    def __init__(self, TH1D_, iTED_, Crystal_, Configuration_, Window_, Calibration_):  

        self.__TH1D = TH1D_
        self.__iTED = iTED_
        self.__Crystal = Crystal_
        self.__Configuration = Configuration_
        self.__Window = Window_
        self.__Calibration = Calibration_
        
    def __call__(self, ch):
        return numpy.polyval(self.__Calibration[::-1],ch)
    
    def __ch__(self, en):
        p = numpy.poly1d(self.__Calibration[::-1])
        temp  = set(i for i in (p - en).roots if i > 0)
        return list(temp).pop()
    
    def TH1D(self):
        return self.__TH1D
    
    def Calibration(self):
        return self.__Calibration
    
    def iTED(self):
        return self.__iTED
    
    def Crystal(self):
        return self.__Crystal
    
    def Configuration(self):
        return self.__Configuration
    
    def Window(self):
        return self.__Window
    
    def Rate(self, Time_):
        return self.__TH1D.Integral()/Time_
    
    def Alpha(self, Time_):
        return self.__TH1D.Integral(
            self.__TH1D.FindBin(self.__ch__(1600)),
            self.__TH1D.FindBin(self.__ch__(2800))
        )/Time_
    
    def __repr__(self):
        return "iTED: {},Crystal: {},Configuration: {},Window: {}".format(
            self.__iTED,
            self.__Crystal,
            self.__Configuration,
            self.__Window
        )
    
    def __str__(self):
        return "{}.{}.{}.{}".format(
            self.__iTED,
            self.__Crystal,
            self.__Configuration,
            self.__Window
        )

In [4]:
iTEDA_cal = pandas.read_csv(
    "../../data/2023-03-02/Energy_Calibrations_02_03_2023/Energy_Calibrations_iTEDA.dat",
    delim_whitespace=True,
    names=["P0","P1","P2"]
)

iTEDB_cal = pandas.read_csv(
    "../../data/2023-03-02/Energy_Calibrations_02_03_2023/Energy_Calibrations_iTEDB.dat",
    delim_whitespace=True,
    names=["P0","P1","P2"]
)

iTEDC_cal = pandas.read_csv(
    "../../data/2023-03-02/Energy_Calibrations_02_03_2023/Energy_Calibrations_iTEDC.dat",
    delim_whitespace=True,
    names=["P0","P1","P2"]
)

iTEDD_cal = pandas.read_csv(
    "../../data/2023-03-02/Energy_Calibrations_02_03_2023/Energy_Calibrations_iTEDD.dat",
    delim_whitespace=True,
    names=["P0","P1","P2"]
)

iTED_cal = pandas.concat([iTEDA_cal.T, iTEDB_cal.T, iTEDC_cal.T, iTEDD_cal.T], axis=1, keys=['A', 'B', 'C', 'D'])

iTED_cal

A                                                          B  \
           0           1           2          3           4           0   
P0  97.91440  138.744000  164.131000  96.575700  184.627000  123.545000   
P1   1.96942    1.813160    2.304510   1.764130    2.327000    2.228220   
P2   0.00042    0.000665    0.001125   0.000595    0.001193    0.000559   

                                                             C              \
             1           2           3           4           0           1   
P0  181.753000  164.939000  162.539000  156.664000  115.328000  147.053000   
P1    2.581200    2.116580    2.010630    2.193700    2.179250    2.005310   
P2    0.001048    0.001154    0.000915    0.000946    0.000469    0.000859   

                                                D                          \
             2           3           4          0           1           2   
P0  129.204000  144.779000  132.346000  91.325100  138.429000  121.824000   
P1    2.079170    2.152760    2.311890   2.016440    2.065850    2.071090   
P2    0.000717    0.000836    0.000898   0.000479    0.000809    0.000667   

                            
             3           4  
P0  138.067000  122.738000  
P1    2.132460    1.964320  
P2    0.000949    0.000705

In [5]:
config = ["888", "887", "88c"]
CW = [100,150,200,250]

ited = ["A","B","C","D"]
crystal = [0,1,2,3,4]

midx = pandas.MultiIndex.from_product(
    [config, CW],
    names=['Configuration', 'Window']
)

mcol = pandas.MultiIndex.from_product(
    [ited, crystal],
    names=['iTED', 'Crystal']
) 

spectra = pandas.DataFrame(index = midx, columns = mcol)

# Access using spectra[iTED,Crystal][Configuration,Window]

In [6]:
iTEDA = {
    "888-100": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137-iTEDA-A2_D.2023_02_23_T.13_40_46_C.itedABCD_lab_2023.02.22_4.0v_888_300s_CW100.root","READ"),
    "888-150": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137-iTEDA-A2_D.2023_02_23_T.13_40_46_C.itedABCD_lab_2023.02.22_4.0v_888_300s_CW150.root","READ"),
    "888-200": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137-iTEDA-A2_D.2023_02_23_T.13_40_46_C.itedABCD_lab_2023.02.22_4.0v_888_300s_CW200.root","READ"),
    "888-250": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137-iTEDA-A2_D.2023_02_23_T.13_40_46_C.itedABCD_lab_2023.02.22_4.0v_888_300s_CW250.root","READ"),
    "88c-100": ROOT.TFile.Open("../../data/Multi_iTED_characterization/CS137-iTEDA-A2_D.2023_03_14_T.14_28_07_C.itedABCD_lab_custom_2023.02.22_4.0v_887_300s_CW100.root","READ"),
    "88c-150": ROOT.TFile.Open("../../data/Multi_iTED_characterization/CS137-iTEDA-A2_D.2023_03_14_T.14_28_07_C.itedABCD_lab_custom_2023.02.22_4.0v_887_300s_CW150.root","READ"),
    "88c-200": ROOT.TFile.Open("../../data/Multi_iTED_characterization/CS137-iTEDA-A2_D.2023_03_14_T.14_28_07_C.itedABCD_lab_custom_2023.02.22_4.0v_887_300s_CW200.root","READ"),
    "88c-250": ROOT.TFile.Open("../../data/Multi_iTED_characterization/CS137-iTEDA-A2_D.2023_03_14_T.14_28_07_C.itedABCD_lab_custom_2023.02.22_4.0v_887_300s_CW250.root","READ"),
    "887-100": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137-iTEDA-A2_D.2023_03_21_T.14_50_48_C.itedABCD_lab_2023.02.22_4.0v_887_300s_CW100.root","READ"),
    "887-150": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137-iTEDA-A2_D.2023_03_21_T.14_50_48_C.itedABCD_lab_2023.02.22_4.0v_887_300s_CW150.root","READ"),
    "887-200": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137-iTEDA-A2_D.2023_03_21_T.14_50_48_C.itedABCD_lab_2023.02.22_4.0v_887_300s_CW200.root","READ"),
    "887-250": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137-iTEDA-A2_D.2023_03_21_T.14_50_48_C.itedABCD_lab_2023.02.22_4.0v_887_300s_CW250.root","READ")
}

iTEDB = {
    "888-100": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137_AfterReconnecting_gum_iTEDB-middle_D.2023_03_02_T.15_22_31_C.itedABCD_lab_2023.02.22_4.0v_888_300s_CW100.root","READ"),
    "888-150": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137_AfterReconnecting_gum_iTEDB-middle_D.2023_03_02_T.15_22_31_C.itedABCD_lab_2023.02.22_4.0v_888_300s_CW150.root","READ"),
    "888-200": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137_AfterReconnecting_gum_iTEDB-middle_D.2023_03_02_T.15_22_31_C.itedABCD_lab_2023.02.22_4.0v_888_300s_CW200.root","READ"),
    "888-250": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137_AfterReconnecting_gum_iTEDB-middle_D.2023_03_02_T.15_22_31_C.itedABCD_lab_2023.02.22_4.0v_888_300s_CW250.root","READ"),
    "88c-100": ROOT.TFile.Open("../../data/Multi_iTED_characterization/CS137-iTEDB-A2_D.2023_03_14_T.14_35_17_C.itedABCD_lab_custom_2023.02.22_4.0v_887_300s_CW100.root","READ"),
    "88c-150": ROOT.TFile.Open("../../data/Multi_iTED_characterization/CS137-iTEDB-A2_D.2023_03_14_T.14_35_17_C.itedABCD_lab_custom_2023.02.22_4.0v_887_300s_CW150.root","READ"),
    "88c-200": ROOT.TFile.Open("../../data/Multi_iTED_characterization/CS137-iTEDB-A2_D.2023_03_14_T.14_35_17_C.itedABCD_lab_custom_2023.02.22_4.0v_887_300s_CW200.root","READ"),
    "88c-250": ROOT.TFile.Open("../../data/Multi_iTED_characterization/CS137-iTEDB-A2_D.2023_03_14_T.14_35_17_C.itedABCD_lab_custom_2023.02.22_4.0v_887_300s_CW250.root","READ"),
    "887-100": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137-iTEDB-A2_D.2023_03_21_T.14_57_39_C.itedABCD_lab_2023.02.22_4.0v_887_300s_CW100.root","READ"),
    "887-150": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137-iTEDB-A2_D.2023_03_21_T.14_57_39_C.itedABCD_lab_2023.02.22_4.0v_887_300s_CW150.root","READ"),
    "887-200": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137-iTEDB-A2_D.2023_03_21_T.14_57_39_C.itedABCD_lab_2023.02.22_4.0v_887_300s_CW200.root","READ"),
    "887-250": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137-iTEDB-A2_D.2023_03_21_T.14_57_39_C.itedABCD_lab_2023.02.22_4.0v_887_300s_CW250.root","READ")
}

iTEDC = {
    "888-100": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137-iTEDC-A1_D.2023_02_23_T.13_26_03_C.itedABCD_lab_2023.02.22_4.0v_888_300s_CW100.root","READ"),
    "888-150": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137-iTEDC-A1_D.2023_02_23_T.13_26_03_C.itedABCD_lab_2023.02.22_4.0v_888_300s_CW150.root","READ"),
    "888-200": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137-iTEDC-A1_D.2023_02_23_T.13_26_03_C.itedABCD_lab_2023.02.22_4.0v_888_300s_CW200.root","READ"),
    "888-250": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137-iTEDC-A1_D.2023_02_23_T.13_26_03_C.itedABCD_lab_2023.02.22_4.0v_888_300s_CW250.root","READ"),
    "88c-100": ROOT.TFile.Open("../../data/Multi_iTED_characterization/CS137-iTEDC-A1_D.2023_03_14_T.14_41_53_C.itedABCD_lab_custom_2023.02.22_4.0v_887_300s_CW100.root","READ"),
    "88c-150": ROOT.TFile.Open("../../data/Multi_iTED_characterization/CS137-iTEDC-A1_D.2023_03_14_T.14_41_53_C.itedABCD_lab_custom_2023.02.22_4.0v_887_300s_CW150.root","READ"),
    "88c-200": ROOT.TFile.Open("../../data/Multi_iTED_characterization/CS137-iTEDC-A1_D.2023_03_14_T.14_41_53_C.itedABCD_lab_custom_2023.02.22_4.0v_887_300s_CW200.root","READ"),
    "88c-250": ROOT.TFile.Open("../../data/Multi_iTED_characterization/CS137-iTEDC-A1_D.2023_03_14_T.14_41_53_C.itedABCD_lab_custom_2023.02.22_4.0v_887_300s_CW250.root","READ"),
    "887-100": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137-iTEDC-A1_D.2023_03_21_T.15_05_52_C.itedABCD_lab_2023.02.22_4.0v_887_300s_CW100.root","READ"),
    "887-150": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137-iTEDC-A1_D.2023_03_21_T.15_05_52_C.itedABCD_lab_2023.02.22_4.0v_887_300s_CW150.root","READ"),
    "887-200": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137-iTEDC-A1_D.2023_03_21_T.15_05_52_C.itedABCD_lab_2023.02.22_4.0v_887_300s_CW200.root","READ"),
    "887-250": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137-iTEDC-A1_D.2023_03_21_T.15_05_52_C.itedABCD_lab_2023.02.22_4.0v_887_300s_CW250.root","READ")
}

iTEDD = {
    "888-100": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137-iTEDD-A1_D.2023_02_23_T.13_03_18_C.itedABCD_lab_2023.02.22_4.0v_888_300s_CW100.root","READ"),
    "888-150": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137-iTEDD-A1_D.2023_02_23_T.13_03_18_C.itedABCD_lab_2023.02.22_4.0v_888_300s_CW150.root","READ"),
    "888-200": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137-iTEDD-A1_D.2023_02_23_T.13_03_18_C.itedABCD_lab_2023.02.22_4.0v_888_300s_CW200.root","READ"),
    "888-250": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137-iTEDD-A1_D.2023_02_23_T.13_03_18_C.itedABCD_lab_2023.02.22_4.0v_888_300s_CW250.root","READ"),
    "88c-100": ROOT.TFile.Open("../../data/Multi_iTED_characterization/CS137-iTEDD-A1_D.2023_03_14_T.14_48_58_C.itedABCD_lab_custom_2023.02.22_4.0v_887_300s_CW100.root","READ"),
    "88c-150": ROOT.TFile.Open("../../data/Multi_iTED_characterization/CS137-iTEDD-A1_D.2023_03_14_T.14_48_58_C.itedABCD_lab_custom_2023.02.22_4.0v_887_300s_CW150.root","READ"),
    "88c-200": ROOT.TFile.Open("../../data/Multi_iTED_characterization/CS137-iTEDD-A1_D.2023_03_14_T.14_48_58_C.itedABCD_lab_custom_2023.02.22_4.0v_887_300s_CW200.root","READ"),
    "88c-250": ROOT.TFile.Open("../../data/Multi_iTED_characterization/CS137-iTEDD-A1_D.2023_03_14_T.14_48_58_C.itedABCD_lab_custom_2023.02.22_4.0v_887_300s_CW250.root","READ"),
    "887-100": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137-iTEDD-A1_D.2023_03_21_T.15_12_02_C.itedABCD_lab_2023.02.22_4.0v_887_300s_CW100.root","READ"),
    "887-150": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137-iTEDD-A1_D.2023_03_21_T.15_12_02_C.itedABCD_lab_2023.02.22_4.0v_887_300s_CW150.root","READ"),
    "887-200": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137-iTEDD-A1_D.2023_03_21_T.15_12_02_C.itedABCD_lab_2023.02.22_4.0v_887_300s_CW200.root","READ"),
    "887-250": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137-iTEDD-A1_D.2023_03_21_T.15_12_02_C.itedABCD_lab_2023.02.22_4.0v_887_300s_CW250.root","READ")
}

In [7]:
for iTED in[0,1,2,3]:
    for Crystal in [0,1,2,3,4]:
        for Configuration in ["888", "887", "88c"]:
            for CW in [100,150,200,250]:
            
                spectra[
                    ["A","B","C","D"][iTED], Crystal
                ][
                    Configuration, CW
                ] = spectrum(
                    [iTEDA,iTEDB,iTEDC,iTEDD][iTED][f"{Configuration}-{CW}"].Get(
                        "{}_{}_amplitude_spectra;1".format(
                            "SCATTERER" if Crystal==0 else "ABSORBER",
                            ["A","B","C","D"][iTED] if Crystal==0 else "{}_{}".format(["A","B","C","D"][iTED],Crystal)
                        )
                    ), 
                    ["A","B","C","D"][iTED], 
                    Crystal, 
                    Configuration, 
                    CW, 
                    iTED_cal[:][["A","B","C","D"][iTED],Crystal]
                )

In [8]:
spectra.dropna(inplace=True)
spectra.applymap(str)

iTED                            A                                         \
Crystal                         0            1            2            3   
Configuration Window                                                       
888           100     A.0.888.100  A.1.888.100  A.2.888.100  A.3.888.100   
              150     A.0.888.150  A.1.888.150  A.2.888.150  A.3.888.150   
              200     A.0.888.200  A.1.888.200  A.2.888.200  A.3.888.200   
              250     A.0.888.250  A.1.888.250  A.2.888.250  A.3.888.250   
887           100     A.0.887.100  A.1.887.100  A.2.887.100  A.3.887.100   
              150     A.0.887.150  A.1.887.150  A.2.887.150  A.3.887.150   
              200     A.0.887.200  A.1.887.200  A.2.887.200  A.3.887.200   
              250     A.0.887.250  A.1.887.250  A.2.887.250  A.3.887.250   
88c           100     A.0.88c.100  A.1.88c.100  A.2.88c.100  A.3.88c.100   
              150     A.0.88c.150  A.1.88c.150  A.2.88c.150  A.3.88c.150   
              200     A.0.88c.200  A.1.88c.200  A.2.88c.200  A.3.88c.200   
              250     A.0.88c.250  A.1.88c.250  A.2.88c.250  A.3.88c.250   

iTED                                         B                            \
Crystal                         4            0            1            2   
Configuration Window                                                       
888           100     A.4.888.100  B.0.888.100  B.1.888.100  B.2.888.100   
              150     A.4.888.150  B.0.888.150  B.1.888.150  B.2.888.150   
              200     A.4.888.200  B.0.888.200  B.1.888.200  B.2.888.200   
              250     A.4.888.250  B.0.888.250  B.1.888.250  B.2.888.250   
887           100     A.4.887.100  B.0.887.100  B.1.887.100  B.2.887.100   
              150     A.4.887.150  B.0.887.150  B.1.887.150  B.2.887.150   
              200     A.4.887.200  B.0.887.200  B.1.887.200  B.2.887.200   
              250     A.4.887.250  B.0.887.250  B.1.887.250  B.2.887.250   
88c           100     A.4.88c.100  B.0.88c.100  B.1.88c.100  B.2.88c.100   
              150     A.4.88c.150  B.0.88c.150  B.1.88c.150  B.2.88c.150   
              200     A.4.88c.200  B.0.88c.200  B.1.88c.200  B.2.88c.200   
              250     A.4.88c.250  B.0.88c.250  B.1.88c.250  B.2.88c.250   

iTED                                                      C               \
Crystal                         3            4            0            1   
Configuration Window                                                       
888           100     B.3.888.100  B.4.888.100  C.0.888.100  C.1.888.100   
              150     B.3.888.150  B.4.888.150  C.0.888.150  C.1.888.150   
              200     B.3.888.200  B.4.888.200  C.0.888.200  C.1.888.200   
              250     B.3.888.250  B.4.888.250  C.0.888.250  C.1.888.250   
887           100     B.3.887.100  B.4.887.100  C.0.887.100  C.1.887.100   
              150     B.3.887.150  B.4.887.150  C.0.887.150  C.1.887.150   
              200     B.3.887.200  B.4.887.200  C.0.887.200  C.1.887.200   
              250     B.3.887.250  B.4.887.250  C.0.887.250  C.1.887.250   
88c           100     B.3.88c.100  B.4.88c.100  C.0.88c.100  C.1.88c.100   
              150     B.3.88c.150  B.4.88c.150  C.0.88c.150  C.1.88c.150   
              200     B.3.88c.200  B.4.88c.200  C.0.88c.200  C.1.88c.200   
              250     B.3.88c.250  B.4.88c.250  C.0.88c.250  C.1.88c.250   

iTED                                                                   D  \
Crystal                         2            3            4            0   
Configuration Window                                                       
888           100     C.2.888.100  C.3.888.100  C.4.888.100  D.0.888.100   
              150     C.2.888.150  C.3.888.150  C.4.888.150  D.0.888.150   
              200     C.2.888.200  C.3.888.200  C.4.888.200  D.0.888.200   
              250     C.2.888.250  C.3.888.250  C.4.888.250  D.0.888.250   
887           100     C.2.887.100  C.3.887.10

In [34]:
def get_resolution(cell):
    
    TH1D = cell.TH1D()
    
    TH1D.GetXaxis().SetRange(TH1D.FindBin(100),TH1D.FindBin(400))
    
    MaxBin   = TH1D.FindBin(TH1D.GetMaximumBin())
    
    ADC_Low  = MaxBin-65
    ADC_High = MaxBin+45

    gaussFit = ROOT.TF1("gaussFit", "pol1(0)+gaus(2)", ADC_Low, ADC_High)
    gaussFit.SetParameters(1000,-1,TH1D.GetMaximum(),MaxBin,10)
    TH1D.Fit(gaussFit,"QR")
    
    sigma = abs(gaussFit.GetParameter(4))
    centroid_ch = gaussFit.GetParameter(3)
    
    sigma_error = gaussFit.GetParError(4)/sigma * 100
    
    x1 = cell(centroid_ch+sigma*numpy.sqrt(2*numpy.log(2)))
    x2 = cell(centroid_ch-sigma*numpy.sqrt(2*numpy.log(2)))
    centroid = cell(centroid_ch)
        
    fwhm = x1-x2
            
    return sigma*numpy.sqrt(2*numpy.log(2))*2/centroid_ch*100, (fwhm/centroid)*100, centroid, sigma_error

In [35]:
def TH1D_draw(cell):
    
    TH1D = cell.TH1D()
    
    canvas = ROOT.TCanvas()
    canvas.cd()
    
    TH1D.SetTitle(repr(cell))
    TH1D.SetStats(False)
    
    latex = ROOT.TLatex()
    latex.SetNDC()
    latex.SetTextSize(0.03)
    
    TH1D.Draw("pe")
    
    l1,l2,l3 = get_resolution(cell)
    
    l4 = uproot.open(f"../../data/nTOF_March2022/888/CW100ns/Resolutions_Cs137_CenterScatter_iTED{cell.iTED()}_8.8.8_100ns.root:grResolEnergy;1").values()[1][cell.Crystal()]
    
    latex.DrawText(0.7, 0.8, "R_ch: {:.2f}%".format(l1))
    
    if l4*0.9 < l2 < l4*1.1:
        latex.DrawText(0.7, 0.75, "R_E: {:.2f}%".format(l2))
    else:
        latex.DrawText(0.7, 0.75, "->R_E: {:.2f}%".format(l2))
        
    if 662*(1-l2/100) < l3 < 662*(1+l2/100):
        latex.DrawText(0.7, 0.7, "E: {:.0f}keV".format(l3))
    else:
        latex.DrawText(0.7, 0.7, "->E: {:.0f}keV".format(l3))
        
    latex.DrawText(0.7, 0.65, "R_E(old): {:.2f}%".format(l4))
    latex.DrawText(0.7, 0.6, "Change: {:.2f}%".format(l2-l4))
            
    return canvas

In [36]:
replaced_5 = {
    100: ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137_iTED_ch_cloudcrystalSD_SiPMSC_D.2023_03_27_T.16_49_18_C.itedABCD_lab_custom_2023.02.22_4.0v_887_300s_CW100.root","READ"),
    150: ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137_iTED_ch_cloudcrystalSD_SiPMSC_D.2023_03_27_T.16_49_18_C.itedABCD_lab_custom_2023.02.22_4.0v_887_300s_CW150.root","READ"),
    200: ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137_iTED_ch_cloudcrystalSD_SiPMSC_D.2023_03_27_T.16_49_18_C.itedABCD_lab_custom_2023.02.22_4.0v_887_300s_CW200.root","READ"),
    250: ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137_iTED_ch_cloudcrystalSD_SiPMSC_D.2023_03_27_T.16_49_18_C.itedABCD_lab_custom_2023.02.22_4.0v_887_300s_CW250.root","READ"),
}

replaced_8 = {
    100: ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137_chSiPMSB_box_D.2023_03_29_T.12_43_44_C.itedABCD_lab_custom_2023.02.22_4.0v_887_300s_CW100.root","READ"),
    150: ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137_chSiPMSB_box_D.2023_03_29_T.12_43_44_C.itedABCD_lab_custom_2023.02.22_4.0v_887_300s_CW150.root","READ"),
    200: ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137_chSiPMSB_box_D.2023_03_29_T.12_43_44_C.itedABCD_lab_custom_2023.02.22_4.0v_887_300s_CW200.root","READ"),
    250: ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137_chSiPMSB_box_D.2023_03_29_T.12_43_44_C.itedABCD_lab_custom_2023.02.22_4.0v_887_300s_CW250.root","READ"),
}

replaced_9 = {
    100: ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137_iTED_regreased_D.2023_03_31_T.11_19_15_C.itedABCD_lab_custom_2023.02.22_4.0v_887_300s_serie1_2_CW100.root","READ"),
    150: ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137_iTED_regreased_D.2023_03_31_T.11_19_15_C.itedABCD_lab_custom_2023.02.22_4.0v_887_300s_serie1_2_CW150.root","READ"),
    200: ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137_iTED_regreased_D.2023_03_31_T.11_19_15_C.itedABCD_lab_custom_2023.02.22_4.0v_887_300s_serie1_2_CW200.root","READ"),
    250: ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137_iTED_regreased_D.2023_03_31_T.11_19_15_C.itedABCD_lab_custom_2023.02.22_4.0v_887_300s_serie1_2_CW250.root","READ"),
}

spectra_1 = spectra.copy(deep=True)

for CW in [100,150,200,250]:
            
    spectra_1["C", 0]["88c", CW] = spectrum(
        replaced_5[CW].Get("SCATTERER_C_amplitude_spectra;1"),
        "C",
        0, 
        "88c", 
        CW, 
        iTED_cal[:]["C",0]
    )
    
    spectra_1["D", 0]["88c", CW] = spectrum(
        replaced_5[CW].Get("SCATTERER_D_amplitude_spectra;1"),
        "D",
        0, 
        "88c", 
        CW, 
        iTED_cal[:]["D",0]
    )
    
    spectra_1["B", 0]["88c", CW] = spectrum(
        replaced_8[CW].Get("SCATTERER_B_amplitude_spectra;1"),
        "B",
        0, 
        "88c", 
        CW, 
        iTED_cal[:]["B",0]
    )
    
    spectra_1["D", 1]["88c", CW] = spectrum(
        replaced_9[CW].Get("ABSORBER_D_1_amplitude_spectra;1"),
        "D",
        1, 
        "88c", 
        CW, 
        iTED_cal[:]["D",1]
    )
    
    spectra_1["D", 2]["88c", CW] = spectrum(
        replaced_9[CW].Get("ABSORBER_D_2_amplitude_spectra;1"),
        "D",
        2, 
        "88c", 
        CW, 
        iTED_cal[:]["D",2]
    )
    
    spectra_1["D", 3]["88c", CW] = spectrum(
        replaced_9[CW].Get("ABSORBER_D_3_amplitude_spectra;1"),
        "D",
        3, 
        "88c", 
        CW, 
        iTED_cal[:]["D",3]
    )
    
    spectra_1["D", 4]["88c", CW] = spectrum(
        replaced_9[CW].Get("ABSORBER_D_4_amplitude_spectra;1"),
        "D",
        4, 
        "88c", 
        CW, 
        iTED_cal[:]["D",4]
    )

## Analysis

### Energy resolution @662keV

#### Using channel values

In [37]:
spectra_1.applymap(lambda x: get_resolution(x)[0]).T.describe().drop(['count', 'std'], axis=0).style.background_gradient(cmap ='YlOrRd',axis=None)

#### Using the current calibration (March 2023, 888, 100ns)

In [38]:
resolution = spectra_1.applymap(lambda x: get_resolution(x)[1])

In [39]:
resolution.T.describe().drop(['count', 'std'], axis=0).style.background_gradient(cmap ='YlOrRd',axis=None)

In [40]:
resolution.style.background_gradient(cmap ='YlOrRd',axis=None)

In [41]:
resolution_e = spectra_1.applymap(lambda x: get_resolution(x)[-1])
resolution_e.style.background_gradient(cmap ='YlOrRd',axis=None)

### Energy of the Cesium peak using the current calibration (March 2023, 888, 100ns)

In [16]:
fit = spectra_1.applymap(lambda x: abs(get_resolution(x)[2]-662)/662*100)

In [17]:
fit.T.describe().drop(['count', 'std'], axis=0).style.background_gradient(cmap ='YlOrRd',axis=None)

### Count rate using integral <250keV (Calibration: March 2023, 888, 100ns)

In [18]:
cps = spectra_1.applymap(lambda x: x.Rate(300))

In [19]:
cps.T.describe().drop(['count', 'std'], axis=0).style.background_gradient(cmap ='YlOrRd',axis=None)

In [20]:
cps.style.background_gradient(cmap ='YlOrRd',axis=None)

### Alpha activity rate using 1600keV < E < 2800keV (Calibration: March 2023, 888, 100ns)

In [21]:
alpha = spectra_1.applymap(lambda x: x.Alpha(300))

In [22]:
alpha.T.describe().drop(['count', 'std'], axis=0).style.background_gradient(cmap ='YlOrRd',axis=None)

In [23]:
alpha.style.background_gradient(cmap ='YlOrRd',axis=None)

## Correlations

In [24]:
df = pandas.concat([resolution,alpha,cps,fit],keys=["Resolution","Alpha","Rate","Fit"]).drop(['888','887'], level=1, axis=0).drop([100,150,200], level=2, axis=0).T

In [25]:
df.corr().style.background_gradient(cmap ='YlOrRd',axis=None)

,,,Resolution,Alpha,Rate,Fit
,,Configuration,88c,88c,88c,88c
,,Window,250,250,250,250
,Configuration,Window,,,,
Resolution,88c,250,1.000000,-0.121757,-0.070816,-0.241556
Alpha,88c,250,-0.121757,1.000000,0.379040,-0.275395
Rate,88c,250,-0.070816,0.379040,1.000000,-0.256763
Fit,88c,250,-0.241556,-0.275395,-0.256763,1.000000


## Draw fits

In [26]:
canvas = spectra_1.applymap(TH1D_draw)
#canvas.applymap(lambda x: type(x).__name__)

In [29]:
for CW in [100,150,200,250]:
    canvas.loc[("88c",CW),("D",2)].Draw()

## Conclusions:

- File sizes:
     - Decreasing the threshold from 888 to 887 results in a .singles.ldat file with x3 the size.
     - Increasing the threshold manually (88c) using 887 as a base results in a .singles.ldat file with 40% of the size.
     - The sizes of 88c and 888 are comparable.
- Energy resolution:
     - Taking the mean value for the 4 iTEDs with the different integration windows, the energy resolution improves only about .1% when going from 888 to 887.
     - Taking the mean value for the 4 iTEDs with the different integration windows, the energy resolution improves only about .15% when going from 888 to 88c.
     - The energy resolution is above 8% for 3 (iTED-A-4, iTED-B-0 and iTED-C-0) for 887 and 2 (iTED-A-4 and iTED-C-0) for 88c using the 250ns.
- Correlations:
     - There doesn't seem to exist any sort of correlation between energy resolution and noise or alpha rate.